In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from addict import Dict

from vmodel.util import color as vcolor

In [ ]:
plt.rcParams['figure.figsize'] = [4, 3.5]
plt.rcParams['figure.autolayout'] = True

In [ ]:
data_dir = '/home/fabian/vmodel_datasets/neighbor_selection/pointmass'

In [ ]:
paths = sorted(glob.glob(data_dir + '/**/merged_*.nc', recursive=True))
len(paths)

In [ ]:
# Read datasets
expdict = Dict()
for path in paths:
    dirname = os.path.dirname(path)
    dist_str, exp = dirname.split(os.path.sep)[-2:]
    dist = int(dist_str.replace('dist', ''))
    ds = xr.open_dataset(path)
    ds['nndist_min'] = ds.nndist.min('agent')
    del ds['nndist']
    ds['nvisible_mean'] = ds.nvisible.mean('agent')
    del ds['nvisible']
    expdict[dist][exp] = ds

In [ ]:
distdict = Dict()
for dist in expdict:
    ds = xr.concat(expdict[dist].values(), 'exp')
    ds.coords['exp'] = list(expdict[dist].keys())
    distdict[dist] = ds

In [ ]:
# Concatenate datasets along distance
ds = xr.concat(distdict.values(), 'dist')
ds.coords['dist'] = list(distdict.keys())

In [ ]:
if 'agent' in ds.coords:
    del ds.coords['agent']

In [ ]:
labels = {
    'metric': 'metric',
    'visual': 'visual',
    'visual_myopic': 'visual + myopic',
    'visual_topo6': 'visual + topological',
    'visual_voronoi': 'visual + voronoi'
}

In [ ]:
colors = {
    'metric': vcolor.metric,
    'visual': vcolor.visual,
    'visual_myopic': vcolor.myopic,
    'visual_topo6': vcolor.topological,
    'visual_voronoi': vcolor.voronoi
}

In [ ]:
timeslice = slice(150, 200)

In [ ]:
dst = ds.isel(time=timeslice).mean('time')

In [ ]:
# Only select dist = 1
dssize = dst.sel(dist=1)

In [ ]:
def plot_size(ax, da):
    for exp in da.exp.data:
        data = da.sel(exp=exp)
        xs, ys, yerrs = data.nagents, data.mean('run'), data.std('run')
        label, color = labels[exp], colors[exp]
        ax.errorbar(xs, ys, yerrs, fmt='-o', capsize=3, label=label, color=color)
    ax.set(xscale='log')
    ax.set(xticks=ds.nagents, xticklabels=ds.nagents.data)
    ax.set(xlabel=r'number of agents $N$')
    ax.grid()
    ax.legend()

## Distance

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.nndist_min)
ax.set(ylabel=r'min. distance $d^\mathrm{min}$ [$m$]')
ax.set(ylim=(0.0, None))
fig.savefig(f'size_nndist_min.pdf')

## Order

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.order)
ax.set(ylabel=r'avg. order $\phi^\mathrm{order}$')
ax.set(ylim=(0.80, None))
fig.savefig(f'size_order.pdf')

## Union

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.union)
ax.set(ylabel=r'avg. union $\phi^\mathrm{union}$')
fig.savefig(f'size_union.pdf')

## Neighbors

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.nvisible_mean)
ax.set(ylabel=r'avg. number of neighbors $N_i$')
ax.set(yscale='log')
ax.set(yticks=dssize.nagents, yticklabels=dssize.nagents.data)
ax.set(ylim=(1, 3000))
ax.legend(loc='upper left')
fig.savefig(f'size_nvisible_mean.pdf')